基于Transformers的命名实体识别

Step1 导入相关包

In [1]:
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

Step2 加载数据集

In [2]:
ner_datasets = load_dataset("peoples_daily_ner",trust_remote_code=True) 
ner_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 4637
    })
})

In [3]:
ner_datasets["train"][0] #这里的5，6都是由'O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'按照映射ID对应的

{'id': '0',
 'tokens': ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}

In [4]:
ner_datasets["train"].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}

In [5]:
label_list = ner_datasets["train"].features["ner_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

Step3 数据集预处理

In [6]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

In [7]:
tokenizer(ner_datasets["train"][0]["tokens"])

{'input_ids': [[101, 3862, 102], [101, 7157, 102], [101, 3683, 102], [101, 6612, 102], [101, 1765, 102], [101, 4157, 102], [101, 1762, 102], [101, 1336, 102], [101, 7305, 102], [101, 680, 102], [101, 7032, 102], [101, 7305, 102], [101, 722, 102], [101, 7313, 102], [101, 4638, 102], [101, 3862, 102], [101, 1818, 102], [101, 511, 102]], 'token_type_ids': [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], 'attention_mask': [[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1]]}

In [8]:
# is_split_into_words=True 是 tokenizer 中的一个参数，用于处理已经预先分好词的文本数据。
tokenizer(ner_datasets["train"][0]["tokens"],is_split_into_words=True)

{'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
res = tokenizer("interesting word") #有的tokenizer并不是以单词作为分词标准，而是用作子词，例如 "inter-", "est-", "ing"，所以出现一个单词多个ID，一般为英文的情况
res

{'input_ids': [101, 10673, 12865, 12921, 8181, 8681, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [10]:
res.word_ids() # 使用word_ids() 将input_ids转换为词ID

[None, 0, 0, 0, 0, 1, None]

In [11]:
# 借助word_ids 实现标签映射
def process_function(examples):# 例句例如 ["John", "lives", "in", "New", "York"]，["ner_tags"] 是对应的标签列表，例如 [1, 0, 0, 3, 4]
    tokenized_exmaples = tokenizer(examples["tokens"], max_length=128, truncation=True, is_split_into_words=True)
    # 例如得到input_ids：101，125，774，235，865，977，102
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        # 第0个样本：ENR得列表
        # 第1个样本：ENR得列表
        # 第2个样本：ENR得列表
        # 第3个样本：ENR得列表
        # 第4个样本：ENR得列表
        word_ids = tokenized_exmaples.word_ids(batch_index=i) #batch_index=i 参数表明处理批次的第几个样本，word_ids(batch_index=0) 返回的是第一个样本的 word_ids 列表
       # None，0，1，2，3，4，None
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_id]) # 这个就是从每个样本得列表中 在反对应ner_tags得标签
        labels.append(label_ids)
        
    tokenized_exmaples["labels"] = labels
    return tokenized_exmaples

    # 解释 调用这种方法的目的是 如果对于英文文本，出现子词的时候，例如playing分成了play和ing 这样input_ids会增长，但是我的ner_tags不变，经过遍历后，我的play和ing分别为0，0
    # label的长度超过了ner_tags，但是始终和我的input_ids一样长，所以就可以训练。

In [12]:
tokenized_datasets = ner_datasets.map(process_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/2319 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4637
    })
})

In [13]:
print(tokenized_datasets["train"][0])

{'id': '0', 'tokens': ['海', '钓', '比', '赛', '地', '点', '在', '厦', '门', '与', '金', '门', '之', '间', '的', '海', '域', '。'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0], 'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0, -100]}


Step4 创建模型

In [14]:
# 对于所有的非二分类任务，切记要指定num_labels，否则就会device错误
model = AutoModelForTokenClassification.from_pretrained("hfl/chinese-macbert-base", num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model.config.num_labels

7

Step5 创建评估函数

In [16]:
seqeval = evaluate.load("seqeval")
seqeval

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [17]:
import numpy as np

def eval_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=-1)

    # 将id转换为原始的字符串类型的标签
    true_predictions = [
        [label_list[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels) 
    ]

    true_labels = [
        [label_list[l] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels) 
    ]

    result = seqeval.compute(predictions=true_predictions, references=true_labels, mode="strict", scheme="IOB2") #严格模式就是必须完全一样才算准确 mode="strict"

    return {"f1": result["overall_f1"]} #这里为什么要自己建立字典，为什么不能像acc_metric = evaluate.load("accuracy")一样返回就行，是因为seqeval函数返回一大堆字典
                                        #在这里自行构建是因为只想单独取F1


# seqeval的输入必须是
#true_labels = [["O", "O", "B-LOC", "I-LOC", "O"],["B-PER", "I-PER", "O"]]
#pred_labels = [["O", "O", "B-LOC", "I-LOC", "O"],["B-PER", "O", "O"]]
#所以在计算的时候才需要将数字转换为字符串

    

Step6 配置训练参数

In [18]:
args = TrainingArguments(
    output_dir="models_for_ner",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    eval_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    logging_steps=50,
    num_train_epochs=3
)

Step7 创建训练器

In [19]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=eval_metric,
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)
)

Step8 模型训练

In [20]:
trainer.train()

  0%|          | 0/981 [00:00<?, ?it/s]

{'loss': 0.2113, 'grad_norm': 1.2516720294952393, 'learning_rate': 4.745158002038736e-05, 'epoch': 0.15}
{'loss': 0.0443, 'grad_norm': 0.5203180909156799, 'learning_rate': 4.490316004077472e-05, 'epoch': 0.31}
{'loss': 0.0405, 'grad_norm': 0.5110564827919006, 'learning_rate': 4.235474006116208e-05, 'epoch': 0.46}
{'loss': 0.0333, 'grad_norm': 0.533626139163971, 'learning_rate': 3.980632008154944e-05, 'epoch': 0.61}
{'loss': 0.0342, 'grad_norm': 0.5301646590232849, 'learning_rate': 3.72579001019368e-05, 'epoch': 0.76}
{'loss': 0.027, 'grad_norm': 0.6514819264411926, 'learning_rate': 3.4709480122324164e-05, 'epoch': 0.92}


  0%|          | 0/19 [00:00<?, ?it/s]

{'eval_loss': 0.020051028579473495, 'eval_f1': 0.947888096544158, 'eval_runtime': 124.8041, 'eval_samples_per_second': 18.581, 'eval_steps_per_second': 0.152, 'epoch': 1.0}
{'loss': 0.0293, 'grad_norm': 0.5307915210723877, 'learning_rate': 3.2161060142711516e-05, 'epoch': 1.07}
{'loss': 0.0152, 'grad_norm': 0.6042858958244324, 'learning_rate': 2.9612640163098882e-05, 'epoch': 1.22}
{'loss': 0.0157, 'grad_norm': 0.5333635210990906, 'learning_rate': 2.7064220183486238e-05, 'epoch': 1.38}
{'loss': 0.0156, 'grad_norm': 0.20879504084587097, 'learning_rate': 2.45158002038736e-05, 'epoch': 1.53}
{'loss': 0.0144, 'grad_norm': 0.3429284989833832, 'learning_rate': 2.196738022426096e-05, 'epoch': 1.68}
{'loss': 0.0137, 'grad_norm': 0.29953446984291077, 'learning_rate': 1.9418960244648318e-05, 'epoch': 1.83}
{'loss': 0.0134, 'grad_norm': 0.44626057147979736, 'learning_rate': 1.6870540265035677e-05, 'epoch': 1.99}


  0%|          | 0/19 [00:00<?, ?it/s]

{'eval_loss': 0.01730617694556713, 'eval_f1': 0.9521196323226779, 'eval_runtime': 122.7159, 'eval_samples_per_second': 18.897, 'eval_steps_per_second': 0.155, 'epoch': 2.0}
{'loss': 0.0083, 'grad_norm': 0.4421904385089874, 'learning_rate': 1.4322120285423038e-05, 'epoch': 2.14}
{'loss': 0.0061, 'grad_norm': 0.5159140229225159, 'learning_rate': 1.1773700305810397e-05, 'epoch': 2.29}
{'loss': 0.0074, 'grad_norm': 0.8011431097984314, 'learning_rate': 9.225280326197758e-06, 'epoch': 2.45}
{'loss': 0.0068, 'grad_norm': 0.3253189027309418, 'learning_rate': 6.676860346585118e-06, 'epoch': 2.6}
{'loss': 0.0074, 'grad_norm': 0.20929251611232758, 'learning_rate': 4.128440366972477e-06, 'epoch': 2.75}
{'loss': 0.006, 'grad_norm': 0.2123628407716751, 'learning_rate': 1.580020387359837e-06, 'epoch': 2.91}


  0%|          | 0/19 [00:00<?, ?it/s]

{'eval_loss': 0.01737246848642826, 'eval_f1': 0.9551870631766479, 'eval_runtime': 93.4724, 'eval_samples_per_second': 24.809, 'eval_steps_per_second': 0.203, 'epoch': 3.0}
{'train_runtime': 2353.8665, 'train_samples_per_second': 26.592, 'train_steps_per_second': 0.417, 'train_loss': 0.02819741215788504, 'epoch': 3.0}


TrainOutput(global_step=981, training_loss=0.02819741215788504, metrics={'train_runtime': 2353.8665, 'train_samples_per_second': 26.592, 'train_steps_per_second': 0.417, 'total_flos': 3940951205762142.0, 'train_loss': 0.02819741215788504, 'epoch': 3.0})

In [21]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

  0%|          | 0/37 [00:00<?, ?it/s]

{'eval_loss': 0.02321714535355568,
 'eval_f1': 0.9494679071012649,
 'eval_runtime': 229.5434,
 'eval_samples_per_second': 20.201,
 'eval_steps_per_second': 0.161,
 'epoch': 3.0}

Step9 模型预测

In [22]:
from transformers import pipeline

In [23]:
model.config.id2label = {idx: label for idx, label in enumerate(label_list)}
model.config

BertConfig {
  "_name_or_path": "hfl/chinese-macbert-base",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  

In [24]:
# 如果模型是基于GPU训练的，那么推理时要指定device
# 对于NER任务，可以指定aggregation_strategy为simple，得到具体的实体的结果，而不是token的结果
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, device=0, aggregation_strategy="simple")

In [30]:
res = ner_pipe("张文俊在北京上班")
res

[{'entity_group': 'PER',
  'score': 0.9995484,
  'word': '张 文 俊',
  'start': 0,
  'end': 3},
 {'entity_group': 'LOC',
  'score': 0.9985008,
  'word': '北 京',
  'start': 4,
  'end': 6}]